In [ ]:
!pip install tensorflow_text

     |████████████████████████████████| 4.9 MB 7.2 MB/s 


In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text
import os
import pandas as pd
import numpy as np

In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
INPUT_DIR = "/content/drive/MyDrive/project_data/all_data_cats_with_gt50"
OUTPUT_DIR = "/content/drive/MyDrive/project_data/all_data_cats_with_gt50/usem"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_train = pd.read_csv(os.path.join(INPUT_DIR, "train.csv"), header=0)
df_train = df_train.dropna(how="any")
df_train["targetcat_encoded"] = df_train["targetcat"].astype('category').cat.codes
df_train = df_train.rename(columns={"targetcat_encoded": "label", "description": "text"})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
col = 'targetcat'
class MyDict(dict):
    def __missing__(self, key):
        return None
mapping_dict = MyDict({k: v for k, v in zip(df_train['targetcat'], df_train['label'])})
mapping_dict[]

{'Шланги': 1614,
 'Вибраторы для бетона': 250,
 'Технические светильники': 1416,
 'Гарантийные сервисы и обслуживание БТ': 291,
 'Мужская парфюмерия': 833,
 'Фотоаппараты': 1543,
 'Затирочные смеси (Фуга)': 467,
 'Удилища': 1477,
 'Электровелосипеды': 1642,
 'Планшеты': 1031,
 'Виски': 259,
 'Наборы по уходу за волосами': 867,
 'Аксессуары к швейной технике': 124,
 'Игровая форма и экипировка': 484,
 'Косметика по уходу для детей': 656,
 'Мобильные телефоны': 801,
 'Сетевые фильтры, адаптеры и удлинители': 1228,
 'Чехлы для планшетов': 1591,
 'Школьные рюкзаки и ранцы': 1612,
 'Стеклоочистители': 1320,
 'Опрыскиватели': 943,
 'Аксессуары для наушников': 88,
 'Аксессуары и комплектующие для паяльного оборудования': 115,
 'Субстраты и грунты для растений': 1346,
 'Беговые дорожки': 172,
 'Шампунь': 1601,
 'Проекторы': 1099,
 'Оборудование для салонов красоты': 927,
 'Краскопульты': 670,
 'Подгузники': 1050,
 'Антисептики для дерева': 144,
 'Офисные сейфы': 966,
 'Научная и техническая ли

In [ ]:
mapping_dict_reverse

{1614: 'Шланги',
 250: 'Вибраторы для бетона',
 1416: 'Технические светильники',
 291: 'Гарантийные сервисы и обслуживание БТ',
 833: 'Мужская парфюмерия',
 1543: 'Фотоаппараты',
 467: 'Затирочные смеси (Фуга)',
 1477: 'Удилища',
 1642: 'Электровелосипеды',
 1031: 'Планшеты',
 259: 'Виски',
 867: 'Наборы по уходу за волосами',
 124: 'Аксессуары к швейной технике',
 484: 'Игровая форма и экипировка',
 656: 'Косметика по уходу для детей',
 801: 'Мобильные телефоны',
 1228: 'Сетевые фильтры, адаптеры и удлинители',
 1591: 'Чехлы для планшетов',
 1612: 'Школьные рюкзаки и ранцы',
 1320: 'Стеклоочистители',
 943: 'Опрыскиватели',
 88: 'Аксессуары для наушников',
 115: 'Аксессуары и комплектующие для паяльного оборудования',
 1346: 'Субстраты и грунты для растений',
 172: 'Беговые дорожки',
 1601: 'Шампунь',
 1099: 'Проекторы',
 927: 'Оборудование для салонов красоты',
 670: 'Краскопульты',
 1050: 'Подгузники',
 144: 'Антисептики для дерева',
 966: 'Офисные сейфы',
 900: 'Научная и техническ

In [ ]:
import joblib
joblib.dump(mapping_dict, os.path.join(OUTPUT_DIR, "mapping_dict"), compress=3)

['/content/drive/MyDrive/project_data/all_data_cats_with_gt50/usem/mapping_dict']

In [ ]:
joblib.dump(mapping_dict_reverse, os.path.join(OUTPUT_DIR, "mapping_dict_reverse"), compress=3)

['/content/drive/MyDrive/project_data/all_data_cats_with_gt50/usem/mapping_dict_reverse']

In [ ]:
df_valid = pd.read_csv(os.path.join(INPUT_DIR, "valid.csv"), header=0)
df_valid = df_valid.dropna(how="any")
df_valid["targetcat_encoded"] = df_valid["targetcat"].map(mapping_dict)
df_valid = df_valid.rename(columns={"targetcat_encoded": "label", "description": "text"})
df_valid = df_valid.dropna(how="any")
df_valid["label"] = df_valid["label"].astype(int)

In [ ]:
df_train.head()

,text,title,targetcat,label
0,Продается в упаковках Цена зависит от количест...,Капельная лента Netafim Streamline Х 6мил - 30...,Шланги,1614
1,Виброуплотнитель бетона Kraissmann 950 ВR 2/35...,Глубинный вибратор Kraissmann 950 ВR 2/35,Вибраторы для бетона,250
2,Компания LEDVANCE появилась на свет в 2016 год...,Светодиодная панель LEDVANCE ECO 40W 4000K 60x...,Технические светильники,1416
3,Твій новий пристрій перестав працювати через м...,Защита от повреждений (65001-100000),Гарантийные сервисы и обслуживание БТ,291
4,Silver Scent Intense – насыщенный аромат с ярк...,Туалетная вода для мужчин Jacques Bogart Silve...,Мужская парфюмерия,833


In [ ]:
df_valid[df_valid.label == 291]

,text,title,targetcat,label
4793,Обычно гарантия производителя на бытовую и циф...,+2 года гарантии (1001-2000),Гарантийные сервисы и обслуживание БТ,291
36050,Твій новий пристрій перестав працювати через м...,Защита от повреждений (40001-65000),Гарантийные сервисы и обслуживание БТ,291
44256,"Модель нового ПК, яка би повністю задовольняла...",Збирання комп'ютера,Гарантийные сервисы и обслуживание БТ,291
46948,Твій новий пристрій перестав працювати через м...,Защита от повреждений (3001-4000),Гарантийные сервисы и обслуживание БТ,291
50819,"Купуєте техніку, а до неї немає гарантії від в...",Гарантія від Support.ua (65001-100000),Гарантийные сервисы и обслуживание БТ,291
58578,Обычно гарантия производителя на бытовую и циф...,+2 года гарантии (5001-7500),Гарантийные сервисы и обслуживание БТ,291
59444,Обычно гарантия производителя на бытовую и циф...,+1 год гарантии (40001-65000),Гарантийные сервисы и обслуживание БТ,291
63915,Обычно гарантия производителя на холодильники ...,+1 год гарантии для холодильников (12001-20000),Гарантийные сервисы и обслуживание БТ,291
73482,Твій новий пристрій перестав працювати через м...,Защита от повреждений (10001-15000),Гарантийные сервисы и обслуживание БТ,291
73808,Обычно гарантия производителя на холодильники ...,Продление гарантии на 2 года для холодильников...,Гарантийные сервисы и обслуживание БТ,291


In [ ]:
hub_layer = hub.KerasLayer(embed, input_shape=[], 
                           dtype=tf.string, trainable=False)

In [ ]:
category_counts = len(df_train['label'].value_counts())
category_counts

1673

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
#model.add(tf.keras.layers.Dense(128, activation='relu'))
#model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(category_counts, activation='softmax'))
model.summary()
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 512)               68927232  
                                                                 
 dense (Dense)               (None, 1673)              858249    
                                                                 
Total params: 69,785,481
Trainable params: 858,249
Non-trainable params: 68,927,232
_________________________________________________________________


In [ ]:
#module_url = 'https://tfhub.dev/google/universal-sentence-encoder-multilingual/3'
#embedding_layer = hub.KerasLayer(module_url)
#hidden_layer = tf.keras.layers.Dense(256, activation='relu')
#output_layer = tf.keras.layers.Dense(category_counts, activation='softmax')

#inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
#x = embedding_layer(tf.squeeze(tf.cast(inputs, tf.string)))
#x = hidden_layer(x)
#outputs = output_layer(x)

#model = tf.keras.Model(inputs=inputs, outputs=outputs)

#model.summary()
#model.compile(optimizer='adam',
#              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
#              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(x=df_train['text'],
                    y=df_train['label'],
                    validation_data = (df_valid['text'], df_valid['label']),
                    epochs=25,
                    batch_size=512,
                    callbacks=[early_stopping],
                    verbose=1)

Epoch 1/25
1015/1015 [==============================] - 636s 620ms/step - loss: 5.3142 - sparse_categorical_accuracy: 0.3134 - val_loss: 3.9303 - val_sparse_categorical_accuracy: 0.4013
Epoch 2/25
1015/1015 [==============================] - 632s 623ms/step - loss: 3.2493 - sparse_categorical_accuracy: 0.4748 - val_loss: 2.7407 - val_sparse_categorical_accuracy: 0.5324
Epoch 3/25
1015/1015 [==============================] - 631s 622ms/step - loss: 2.4063 - sparse_categorical_accuracy: 0.5742 - val_loss: 2.1666 - val_sparse_categorical_accuracy: 0.6036
Epoch 4/25
1015/1015 [==============================] - 631s 622ms/step - loss: 1.9612 - sparse_categorical_accuracy: 0.6316 - val_loss: 1.8391 - val_sparse_categorical_accuracy: 0.6488
Epoch 5/25
1015/1015 [==============================] - 632s 623ms/step - loss: 1.6912 - sparse_categorical_accuracy: 0.6681 - val_loss: 1.6311 - val_sparse_categorical_accuracy: 0.6777
Epoch 6/25
1015/1015 [==============================] - 632s 622ms/ste

In [ ]:
OUTPUT_DIR = "/content/drive/MyDrive/project_data/all_data_cats_with_gt50/usem"
model.save(OUTPUT_DIR)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/project_data/all_data_cats_with_gt50/usem/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/project_data/all_data_cats_with_gt50/usem/assets


In [ ]:
print(tf.__version__)

2.7.0


In [ ]:
model.save(os.path.join(OUTPUT_DIR , 'keras_model/my_model.h5'))

In [32]:
df_test = pd.read_csv(os.path.join(INPUT_DIR, "test.csv"), header=0)
df_test = df_valid.dropna(how="any")
df_test["targetcat_encoded"] = df_test["targetcat"].map(mapping_dict)
model.evaluate(x=df_test['text'], y=df_test['targetcat_encoded'], batch_size=512)

170/170 [==============================] - 94s 555ms/step - loss: 0.9269 - sparse_categorical_accuracy: 0.7780


[0.926925003528595, 0.7780344486236572]